In [103]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd

In [104]:
df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Midterm\Midterm Practice\sp500_data.csv")

In [116]:
df[df["Location of Headquarters"] == "New York, New York"]

,Ticker symbol,Company,GICS Sector,Location of Headquarters,Price,PercentReturn
1,AXP,American Express Co,Financials,"New York, New York",49.23,1.53
3,AVP,Avon Products,Consumer Staples,"New York, New York",18.31,5.26
8,CBS,CBS Corp.,Consumer Discretionary,"New York, New York",28.64,1.47
32,JPM,JPMorgan Chase & Co.,Financials,"New York, New York",37.66,3.01
60,TIF,Tiffany & Co.,Consumer Discretionary,"New York, New York",63.28,1.75
64,VZ,Verizon Communications,Telecommunications Services,"New York, New York",37.79,4.99


In [105]:
# Extract the Price and PercentReturn values
prices = df['Price'].tolist()
percent_returns = df['PercentReturn'].tolist()
# Extract sector and location information
sectors = df['GICS Sector'].tolist()
locations = df['Location of Headquarters'].tolist()

In [106]:
# Create a new model
model = gb.Model("Investment Portfolio")

In [107]:
# Decision variables
x = model.addVars(67, lb=0, vtype=GRB.CONTINUOUS, name="Investment_Amount")

# Objective function
model.setObjective(gb.quicksum((percent_returns[i]/100) * x[i] for i in range(67)), GRB.MAXIMIZE)

In [108]:
# Total investment amount
total_investment = 10000000
max_investment_per_stock = 600000

In [109]:
# Constraints
# Total investment constraint
model.addConstr(gb.quicksum(x[i] for i in range(67)) == total_investment, "Total_Investment")

# Individual stock investment constraint

model.addConstrs(x[i] <= max_investment_per_stock for i in range(67))

# Sector investment constraints
telecom_indices = [i for i, sector in enumerate(sectors) if sector == "Telecommunications Services"]
it_indices = [i for i, sector in enumerate(sectors) if sector == "Information Technology"]
discretionary_indices = [i for i, sector in enumerate(sectors) if sector == "Consumer Discretionary"]
staples_indices = [i for i, sector in enumerate(sectors) if sector == "Consumer Staples"]
energy_indices = [i for i, sector in enumerate(sectors) if sector == "Energy"]


model.addConstr(gb.quicksum(x[i] for i in telecom_indices) <= 500000, "Telecom_Investment")
model.addConstr(gb.quicksum(x[i] for i in it_indices) >= 0.75 * gb.quicksum(x[i] for i in telecom_indices), "IT_Investment")
model.addConstr(gb.quicksum(x[i] for i in energy_indices) >= 1000000, "Energy_Investment")

# Consumer sectors difference constraint
model.addConstr(gb.quicksum(x[i] for i in discretionary_indices) - gb.quicksum(x[i] for i in staples_indices) <= 200000, "Consumer_Difference_Upper")
model.addConstr(gb.quicksum(x[i] for i in staples_indices) - gb.quicksum(x[i] for i in discretionary_indices) <= 200000, "Consumer_Difference_Lower")

# Location-based investment constraint
ny_indices = [i for i, location in enumerate(locations) if location == "New York, New York"]
model.addConstr(gb.quicksum(x[i] for i in ny_indices) >= 300000, "NY_Investment")

<gurobi.Constr *Awaiting Model Update*>

In [117]:
# Optimally solve the problem
model.optimize()

print(model.printAttr('X'))

# Value of the objective function
print("Expected 1-year Return: ", round(model.objVal, 2))

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 74 rows, 67 columns and 196 nonzeros
Coefficient statistics:
  Matrix range     [8e-01, 1e+00]
  Objective range  [2e-03, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+05, 1e+07]

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  5.134600000e+05

    Variable            X 
-------------------------
Investment_Amount[0]       600000 
Investment_Amount[3]       600000 
Investment_Amount[6]       400000 
Investment_Amount[11]       600000 
Investment_Amount[16]       600000 
Investment_Amount[19]       600000 
Investment_Amount[21]       400000 
Investment_Amount[23]       500000 
Investment_Amount[27]       600000 
Investment_Amount[34]       600000 
Investment_Amount[39]       600

In [118]:
# Print the investment amounts and company headquarters
print("Investment Portfolio:")
for i in range(67):
    if x[i].x > 0:
        print(f"Company {i+1}: ${x[i].x}, Headquarters: {locations[i]}")

print(f"-"*50)
for constr in model.getConstrs():
        print(f"Constraint: {constr.ConstrName}, Shadow Price: {constr.Pi}")

Investment Portfolio:
Company 1: $600000.0, Headquarters: St. Louis, Missouri
Company 4: $600000.0, Headquarters: New York, New York
Company 7: $400000.0, Headquarters: Bethpage, New York
Company 12: $600000.0, Headquarters: Houston, Texas
Company 17: $600000.0, Headquarters: Chicago, Illinois
Company 20: $600000.0, Headquarters: Charlotte, North Carolina
Company 22: $400000.0, Headquarters: Irving, Texas
Company 24: $500000.0, Headquarters: Stamford, Connecticut
Company 28: $600000.0, Headquarters: Long Beach, California
Company 35: $600000.0, Headquarters: New Hyde Park, New York
Company 40: $600000.0, Headquarters: Whitehouse Station, New Jersey
Company 41: $375000.0, Headquarters: Chandler, Arizona
Company 46: $600000.0, Headquarters: Bridgeport, Connecticut
Company 47: $600000.0, Headquarters: Washington D.C
Company 50: $600000.0, Headquarters: San Francisco, California
Company 51: $525000.0, Headquarters: Phoenix, Arizona
Company 52: $600000.0, Headquarters: Seattle, Washington
C

In [119]:
# Print the sensitivity analysis report for the objective function coefficients
if model.status == GRB.OPTIMAL:
    print("Sensitivity Analysis Report for Objective Function Coefficients:")
    print("{:<15} {:<15} {:<15} {:<15}".format("Company", "Coefficient", "Allowable Decrease", "Allowable Increase"))
    for i in range(67):
        print("{:<15} {:<15.2f} {:<15.2f} {:<15.2f}".format(f"Company {i+1}", percent_returns[i], x[i].SAObjLow, x[i].SAObjUp))

Sensitivity Analysis Report for Objective Function Coefficients:
Company         Coefficient     Allowable Decrease Allowable Increase
Company 1       4.83            0.04            inf            
Company 2       1.53            -inf            0.04           
Company 3       5.82            -inf            0.15           
Company 4       5.26            0.05            inf            
Company 5       1.74            -inf            0.05           
Company 6       1.89            -inf            0.04           
Company 7       4.22            0.04            0.04           
Company 8       3.49            -inf            0.05           
Company 9       1.47            -inf            0.04           
Company 10      2.69            -inf            0.05           
Company 11      2.02            -inf            0.05           
Company 12      3.62            0.02            inf            
Company 13      2.14            -inf            0.04           
Company 14      0.90            -